In [1]:
# importar paquetes a ocupar
import brawlstats
import pandas as pd
import datetime
import ipywidgets as widgets
import zipfile
import os
import api_key

In [2]:
# crear cliente
client = brawlstats.Client(api_key.api_key)

In [3]:
# importar brawlers
brawlers = pd.DataFrame()

json_brawlers = client.get_brawlers().raw_data

brawlers = pd.concat([brawlers, pd.json_normalize(json_brawlers)])

In [4]:
# reset brawler index y export de dataset
brawlers.reset_index(drop=True, inplace=True)

brawlers.to_csv('datasets/brawlers/brawlers.csv')

print(brawlers.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          64 non-null     int64 
 1   name        64 non-null     object
 2   starPowers  64 non-null     object
 3   gadgets     64 non-null     object
dtypes: int64(1), object(3)
memory usage: 2.1+ KB
None


In [5]:
# import información adicional de brawlers
brawlers_classification = pd.read_csv('datasets/brawlers/brawlers_classification.csv', index_col=0)

In [6]:
# merge de ambos dataframes
brawlerStats = pd.merge(brawlers, brawlers_classification, on='id')
print(brawlerStats.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64 entries, 0 to 63
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              64 non-null     int64  
 1   name            64 non-null     object 
 2   starPowers      64 non-null     object 
 3   gadgets         64 non-null     object 
 4   Brawler         64 non-null     object 
 5   Health          64 non-null     float64
 6   Rarity          64 non-null     object 
 7   Class           64 non-null     object 
 8   Movement_Speed  64 non-null     object 
 9   Range           63 non-null     object 
 10  Reload          63 non-null     object 
 11  Speed_Num       64 non-null     int64  
 12  Range_Num       64 non-null     float64
 13  Reload_Num      63 non-null     float64
 14  Role            64 non-null     object 
dtypes: float64(3), int64(2), object(10)
memory usage: 8.0+ KB
None


In [7]:
# export dataframe final brawlers
brawlerStats.to_csv('datasets/brawlers/brawlers_stats.csv')

print('dimensiones brawlerStats: ' + str(brawlerStats.shape))

dimensiones brawlerStats: (64, 15)


In [8]:
countryCode = ['US','MX','BR','GB','CA','DE','FR','ES','IT','RU','TR','AR','PL','CO','IN','ID','UA','AU','NL','JP','KR','CZ','CH','PH','MY']#,'VN','IE','TH','IL','NO','FI','PT','AT','GR','HU','SG','SA','AE','SE','DK','BZ','CR','GT','HN','NI','PA','SV','BO','CL','EC']
len(countryCode)

25

In [9]:
# sacar el player tag de los top 200 players
topplayer_tag=[]

# top global
leaderboard = client.get_rankings(ranking='players')
for i in leaderboard:
    topplayer_tag.append(i.tag)

# top por regiones en countryCode
for i, item in enumerate(countryCode):
   leaderboard = client.get_rankings(ranking='players',region=item)
   for i in leaderboard:
       topplayer_tag.append(i.tag)

topplayer_tag = list(set(topplayer_tag))

print('cantidad top player tag: ' + str(len(topplayer_tag)))

cantidad top player tag: 5035


In [10]:
# importar battelog de api
progress = widgets.IntProgress(
	value=0, 
	min=0, 
	max=100, 
	description='cargando:',
	style={'bar_color': 'maroon'}) 

processed = widgets.BoundedFloatText(
    value=0,
    min=0,
    max=len(topplayer_tag),
    description='procesado:',
    disabled=False
)

display(progress)
display(processed)

data = {}

for i in range(len(topplayer_tag)):
    json_battlelog = {}
    playertag = topplayer_tag[i]
    try:
        json_battlelog = client.get_battle_logs(playertag).raw_data
    except:
        print("No se pudo recuperar battlelog de tag " + playertag)
    
    for k in range(len(json_battlelog)):
        loaded_json = json_battlelog[k]
        loaded_json['playertag'] = playertag
        data[str(i) + '-' + str(k)] = loaded_json

        progress.value = ((i+1) / len(topplayer_tag)) * 100
        processed.value = i+1

battlelog = pd.DataFrame.from_dict(data, orient='index').reset_index(drop=True)

IntProgress(value=0, description='cargando:', style=ProgressStyle(bar_color='maroon'))

BoundedFloatText(value=0.0, description='procesado:', max=5035.0)

No se pudo recuperar battlelog de tag #2JU2UQYJP
No se pudo recuperar battlelog de tag #CUQJ8CL


In [11]:
# creación del dataframe
#battlelog = pd.DataFrame()

#def split_json(s):
#	# separa un string por el item "
#	s = str(s).split('"')
#
#	return s

#def clean_json(jsonitem):
#	# para una lista de artiuclos de json, separa en " y limpia los elementos impares de la lista resultante 
#	for i in range(len(jsonitem)):
#		splitted = split_json(jsonitem[i])
#		for j in range(len(splitted)):
#			splitted[j] = str(splitted[j]).replace('None','"None"').replace('False','"False"').replace('True','"True"')
#			if j % 2 == 1 and len(splitted[j]) < 20:
#				splitted[j] = '"' + str(splitted[j]).replace("'",'`') + '"'
#			else:
#				splitted[j] = str(splitted[j]).replace("'",'"')
#		jsonitem[i] = ''.join(splitted)
#	
#	return jsonitem

#def cleanjson(val):
#	result = str(val).replace('"',"`").replace(': `', ": '").replace('`,',"',").replace('`}',"'}").replace("e's",'e`s').replace('"ll','`ll').replace("I'm", 'I`m').replace("i'm", 'i`m').replace("' ","` ").replace("t's","t`s").replace('None',"'None'").replace("'",'"')
#	return result

# progress = widgets.IntProgress(
# 	value=0, 
# 	min=0, 
# 	max=100, 
# 	description='cargando:',
# 	style={'bar_color': 'maroon'}) 

# processed = widgets.BoundedFloatText(
#     value=0,
#     min=0,
#     max=len(topplayer_tag),
#     description='procesado:',
#     disabled=False
# )

# display(progress)
# display(processed)

# for i in range(len(topplayer_tag)):

# 	json_battlelog = {}
# 	playertag = topplayer_tag[i]
# 	try:
# 		json_battlelog = client.get_battle_logs(playertag).raw_data
# 	except:
# 		print("No se pudo recuperar battlelog de tag " + playertag)

# 	for k in range(len(json_battlelog)):
# 		loaded_json = json_battlelog[k]
# 		json_battlelog[k]['playertag'] = playertag
# 		try:
# 			battlelog = battlelog.append(pd.json_normalize(loaded_json), ignore_index=True)
# 		except:
# 			print("no se pudo importar " + playertag + " battlelog numero " + str(k))

# 	progress.value = ((i+1) / len(topplayer_tag)) * 100
# 	processed.value = i


In [12]:
# desglose de dataframe
battlelog = pd.merge(left=battlelog, right=pd.json_normalize(battlelog['event']).add_prefix('event.'), left_index=True, right_index=True)
battlelog = pd.merge(left=battlelog, right=pd.json_normalize(battlelog['battle']).add_prefix('battle.'), left_index=True, right_index=True)

In [13]:
# reset battlelog index
battlelog.drop('event', inplace=True, axis=1)
battlelog.drop('battle', inplace=True, axis=1)
print('dimensiones battlelog: ' + str(battlelog.shape))

dimensiones battlelog: (115989, 28)


In [14]:
battlelog.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115989 entries, 0 to 115988
Data columns (total 28 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   battleTime                          115989 non-null  object 
 1   playertag                           115989 non-null  object 
 2   event.id                            115989 non-null  int64  
 3   event.mode                          98719 non-null   object 
 4   event.map                           98719 non-null   object 
 5   battle.mode                         115989 non-null  object 
 6   battle.type                         115100 non-null  object 
 7   battle.result                       84587 non-null   object 
 8   battle.duration                     83800 non-null   float64
 9   battle.trophyChange                 75528 non-null   float64
 10  battle.teams                        97410 non-null   object 
 11  battle.starPlayer.tag     

In [15]:
# cuenta tipos de juego
battlelog['battle.type'].value_counts()

ranked        89830
soloRanked    11191
teamRanked     6745
friendly       4955
challenge      2132
tournament      247
Name: battle.type, dtype: int64

In [16]:
# eliminar friendly
battlelog = battlelog.loc[battlelog['battle.type'] != "friendly"]

In [17]:
# cuenta modos de juego
battlelog['battle.mode'].value_counts() 

brawlBall       31400
soloShowdown    16783
duoShowdown     13466
knockout        10553
bounty          10394
gemGrab         10031
heist            9760
hotZone          7758
bossFight         799
roboRumble         60
bigGame            30
Name: battle.mode, dtype: int64

In [18]:
# eliminar modos alt y modos showdown
modos_alt = ['bossFight','roboRumble','bigGame','soloShowdown','duoShowdown']
battlelog = battlelog.loc[~battlelog['battle.mode'].isin(modos_alt)]

In [19]:
# eliminar event vacio
battlelog['event.mode'] = battlelog['event.mode'].fillna('unknown')
battlelog = battlelog.loc[battlelog['event.mode'] != "unknown"]

In [20]:
# eliminar map vacio
battlelog['event.map'] = battlelog['event.map'].fillna('unknown')
battlelog = battlelog.loc[battlelog['event.map'] != "unknown"]

In [21]:
# reset battlelog index
battlelog.reset_index(drop=True, inplace=True)

print('dimensiones battlelog: ' + str(battlelog.shape))

dimensiones battlelog: (65434, 28)


In [22]:
# crear listado de players en battelog

#playerlist = pd.DataFrame(pd.concat([
#	battlelog['playertag'], 
#	battlelog['battle.starPlayer.tag'], 
#	battlelog['battle.team1.player1.tag'], 
#	battlelog['battle.team1.player2.tag'], 
#	battlelog['battle.team1.player3.tag'], 
#	battlelog['battle.team2.player1.tag'], 
#	battlelog['battle.team2.player2.tag'], 
#	battlelog['battle.team2.player3.tag']
#	], ignore_index=True).drop_duplicates().reset_index(drop=True))
#
#playerlist.shape

In [23]:
# crear listado nuevos playersplayers_hist
#playerlist_merge = pd.merge(playerlist, players_hist['tag'], left_on=0, right_on='tag', how='left').drop_duplicates().reset_index(drop=True)
#
#playerlist_final = playerlist_merge[0][playerlist_merge['tag'].isna()].drop_duplicates().reset_index(drop=True)
#
#playerlist_final.shape

In [24]:
# agregar datos de player
#players = pd.DataFrame()
#
#for i in range(len(playerlist_final)):
#
#	json_player = []
#	playertag = playerlist_final.loc[i]
#	try:
#		json_player = client.get_profile(playerlist_final.loc[i]).raw_data
#		del json_player['brawlers']
#	except:
#		print("No se pudo recuperar player de tag " + str(playertag))
#
#	try:
#		players = pd.concat([players, pd.json_normalize(json_player)])
#	except:
#		print("no se pudo importar player tag " + str(playertag))

# progress = widgets.IntProgress(
# 	value=0, 
# 	min=0, 
# 	max=100, 
# 	description='cargando:',
# 	style={'bar_color': 'maroon'}) 

# processed = widgets.BoundedFloatText(
#     value=0,
#     min=0,
#     max=len(topplayer_tag),
#     description='procesado:',
#     disabled=False
# )

# display(progress)
# display(processed)

# timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")

# player = {}
# json_player = []

# for i, playertag in enumerate(topplayer_tag):
#     try:
#         json_player = client.get_profile(playertag).raw_data
#         del json_player['brawlers']
#         json_player['import_date'] = timestamp
#         player[str(i)] = json_player
#     except:
#         print("No se pudo recuperar profile de tag " + str(playertag))

#     progress.value = ((i+1) / len(topplayer_tag)) * 100
#     processed.value = i+1

# players = pd.DataFrame.from_dict(player, orient='index').reset_index(drop=True)

In [25]:
# players_hist = pd.read_csv('datasets/players/players.csv', index_col=0)
# players_hist.shape

In [26]:
# concatenar las bases
# players = pd.concat([players, players_hist], ignore_index=True).drop_duplicates(['tag']).reset_index(drop=True)

# print('dimensiones players: ' + str(players.shape))

In [27]:
# players.to_csv('datasets/players/players.csv')

In [28]:
# descomponer la columna teams

# def battle_delimiter(i, j):
# 	result = pd.json_normalize(pd.json_normalize(pd.json_normalize(pd.json_normalize(rawdata['battle'])['teams'])[i])[j])
# 	return result

def normalize_to_df(i, t, p):
	battlelog.loc[i,'battle.team' + str(t) + '.player' + str(p) + '.tag'] = normalized[t - 1][p - 1]['tag']
	battlelog.loc[i,'battle.team' + str(t) + '.player' + str(p) + '.name'] = normalized[t - 1][p - 1]['name']
	battlelog.loc[i,'battle.team' + str(t) + '.player' + str(p) + '.brawler.id'] = normalized[t - 1][p - 1]['brawler.id']
	battlelog.loc[i,'battle.team' + str(t) + '.player' + str(p) + '.brawler.name'] = normalized[t - 1][p - 1]['brawler.name']
	battlelog.loc[i,'battle.team' + str(t) + '.player' + str(p) + '.brawler.power'] = normalized[t - 1][p - 1]['brawler.power']
	battlelog.loc[i,'battle.team' + str(t) + '.player' + str(p) + '.brawler.trophies'] = normalized[t - 1][p - 1]['brawler.trophies']

normalized = pd.DataFrame()

for i, team in enumerate(battlelog['battle.teams']):
	if team != None:
		try:
			normalized = pd.json_normalize(team, errors='ignore').transpose()
			normalize_to_df(i, 1, 1)
			normalize_to_df(i, 1, 2)
			normalize_to_df(i, 1, 3)
			normalize_to_df(i, 2, 1)
			normalize_to_df(i, 2, 2)
			normalize_to_df(i, 2, 3)
		except:
			print("no se pudo transponer")

battlelog.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65434 entries, 0 to 65433
Data columns (total 64 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   battleTime                             65434 non-null  object 
 1   playertag                              65434 non-null  object 
 2   event.id                               65434 non-null  int64  
 3   event.mode                             65434 non-null  object 
 4   event.map                              65434 non-null  object 
 5   battle.mode                            65434 non-null  object 
 6   battle.type                            65434 non-null  object 
 7   battle.result                          65434 non-null  object 
 8   battle.duration                        65434 non-null  float64
 9   battle.trophyChange                    48559 non-null  float64
 10  battle.teams                           65434 non-null  object 
 11  ba

In [29]:
# eliminar battle teams
battlelog = battlelog.drop(columns=[
'battle.teams'
])

In [30]:
# traer archivo histórico battlelog
with zipfile.ZipFile('datasets/teams/battlelog_teams.zip', 'r') as zip_ref:
    # Extraer el archivo CSV
    zip_ref.extractall('datasets/teams/')
    
battlelog_hist = pd.read_csv('datasets/teams/battlelog_teams.csv', index_col=0)

print('dimensiones battlelog hist: ' + str(battlelog_hist.shape))

C:\Users\alniquia\AppData\Local\Temp\ipykernel_12296\612617333.py:6: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  battlelog_hist = pd.read_csv('datasets/teams/battlelog_teams.csv', index_col=0)


dimensiones battlelog hist: (484473, 63)


In [31]:
# agregar nuevos reg a histórico
battlelog = pd.concat([battlelog, battlelog_hist])
print('dimensiones battlelog concat: ' + str(battlelog.shape))

dimensiones battlelog concat: (549907, 63)


In [32]:
# eliminar battelogs duplicados
battlelog = battlelog.drop_duplicates(['battleTime', 'event.id', 'event.mode', 'event.map', 'battle.mode', 'battle.type', 'battle.duration', 'battle.team1.player1.tag'], ignore_index=True)

print('dimensiones battlelog final: ' + str(battlelog.shape))

dimensiones battlelog final: (531208, 63)


In [33]:
# export dataset teams completo mas histórico
battlelog.to_csv('datasets/teams/battlelog_teams.csv')

# Comprimir el archivo CSV
with zipfile.ZipFile('datasets/teams/battlelog_teams.zip', 'w') as zip_file:
    zip_file.write('datasets/teams/battlelog_teams.csv', arcname='battlelog_teams.csv',compress_type=zipfile.ZIP_DEFLATED)

os.remove('datasets/teams/battlelog_teams.csv')

In [34]:
maplist = battlelog[['event.mode','event.map']].drop_duplicates()

maplist.to_csv('datasets/maps/maplist.csv')